In [1]:
import tensorflow as tf

In [2]:

daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)
    

1512
1584
2232
2304
2448
2232


In [9]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2232
1584
2448
2304
2232


In [8]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 1584]
[2232 2304]
[2448 2232]


In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 1584]
[2232 2448]
[2304 2232]


In [20]:
images_ds = tf.data.Dataset.list_files('img/*/*', shuffle=False)

In [21]:
image_count = len(images_ds)
image_count

62

In [25]:
for file in images_ds.take(2):
    print(file.numpy())

b'img\\apples\\0a5ae820-7051-4495-bcca-61bf02897472.jpg'
b'img\\apples\\1613180842.jpg'


In [27]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'img\\banana\\BANANANA.jpg'
b'img\\apples\\Apples-Galore-What-to-Do-scaled.jpg'
b'img\\banana\\banana-fruit-2d-illustration-png.png'


In [28]:
class_names = ["cat","dog"]

In [58]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [30]:
len(train_ds)

49

In [49]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [50]:
get_label("images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [51]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [57]:
img, label = process_image("img\\apples\\0a5ae820-7051-4495-bcca-61bf02897472.jpg")
img.numpy()[:2]

array([[[255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 2

In [59]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [60]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[124.703125 131.04297   95.73047 ]
  [160.875    177.0625   112.70703 ]
  [146.07422  169.07422   98.69922 ]
  ...
  [161.17578  177.6875   167.17578 ]
  [107.58203  142.01953  103.70703 ]
  [ 90.375    139.1875    49.429688]]

 [[ 97.03906   95.20703   75.78906 ]
  [148.0664   158.9961    94.6875  ]
  [129.125    141.46484   81.5625  ]
  ...
  [109.6875   133.125    107.125   ]
  [ 95.125    125.875     89.37891 ]
  [ 91.48047  132.60156   67.29297 ]]

 [[101.13672   99.13672   76.11328 ]
  [149.375    150.96484  105.125   ]
  [139.8086   145.8125   107.50391 ]
  ...
  [ 97.9375   115.9375    89.9375  ]
  [100.96484  116.91016   83.3125  ]
  [102.47266  132.44922   82.64844 ]]

 ...

 [[ 79.17578   81.17578   67.17578 ]
  [ 80.9375    95.0625    76.5     ]
  [ 86.9375   103.9375    85.9375  ]
  ...
  [149.875    173.875    125.875   ]
  [136.34766  157.47266  108.47266 ]
  [125.3125   141.3125    94.3125  ]]

 [[ 74.        75.        61.      ]
  [ 80.125     87.125

In [61]:
def scale(image, label):
    return image/255, label

In [62]:
train_ds = train_ds.map(scale)

In [63]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0. 0. 0. 0.]
****Label:  b'banana'
****Image:  [1. 1. 1.]
****Label:  b'banana'
****Image:  [0. 0. 0. 0.]
****Label:  b'banana'
****Image:  [0.5223652  0.5890319  0.69099265]
****Label:  b'banana'
****Image:  [0.39215687 0.40392157 0.4627451 ]
****Label:  b'apples'
